# Baseline Algorithms for Forest Cover Type Predictions


Data source: https://www.kaggle.com/c/forest-cover-type-prediction

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tools.plotting import scatter_matrix
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

## Load Data

In [ ]:
forest = pd.read_csv("data/train.csv", index_col=0)
original_fields = forest.columns.tolist()
forest.head()

## Create Functions for Feature Engineering

In [ ]:
def labelSoilType(row):
    """
    Label soil types
    """
    for i in range(len(row)):
        if row[i] == 1:
            return 'Soil_Type'+str(i)
        
def azimuth_to_abs(x):
    """
    Only care about the absolute angle from 0 w/o respect to direction
    """
    if x>180:
        return 360-x
    else:
        return x
    
def removeColumns(df):
    to_remove = [] # features to drop
    for c in df.columns.tolist():
        if df[c].std() == 0:
            to_remove.append(c)
    df = df.drop(to_remove, 1)
    print("Dropped the following columns: \n")
    for r in to_remove:
        print (r)
    return df, to_remove

## Feature Engineering

In [ ]:
# Create Soil Type Buckets
soil_types= ['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7',
       'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
       'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
       'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
       'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
       'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
       'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40']
soil_data = pd.read_csv('soil_types.csv').set_index('Soil Type')
forest['Soil Type'] = forest[soil_types+['Cover_Type']].apply(lambda row: labelSoilType(row), axis=1)

#Merge soil data
forest = pd.merge(forest, soil_data, how='left', left_on='Soil Type', right_index=True)
del forest['Soil Type'] # Delete string column

#Remove unnecessary columns and update soil list
forest, removed = removeColumns(forest)
for i in removed:    
    for j in soil_types:
        if i == j:
            soil_types.pop(soil_types.index(j))
    for k in original_fields:
        if i == k:
            original_fields.pop(original_fields.index(k))

# Create feature to that transforms azimuth to its absolute value
forest['Aspect2'] = forest.Aspect.map(azimuth_to_abs)
forest['Aspect2'].astype(int)

# Create feature that determines if the patch is above sea level
forest['Above_Sealevel'] = (forest.Vertical_Distance_To_Hydrology>0).astype(int)

# Bin the Elevation Feature: check the feature exploration notebook for motivation
bins = [0, 2600, 3100, 8000]
group_names = [1, 2, 3]
forest['Elevation_Bucket'] = pd.cut(forest['Elevation'], bins, labels=group_names)
forest['Elevation_0_2600'] = np.where(forest['Elevation_Bucket']== 1, 1, 0)
forest['Elevation_2600_3100'] = np.where(forest['Elevation_Bucket']== 2, 1, 0)
forest['Elevation_3100_8000'] = np.where(forest['Elevation_Bucket']== 3, 1, 0)
forest['Elevation_0_2600'].astype(int)
forest['Elevation_2600_3100'].astype(int)
forest['Elevation_3100_8000'].astype(int)
del forest['Elevation_Bucket']

# Create a feature for no hillshade at 3pm
forest['3PM_0_Hillshade'] = (forest.Hillshade_3pm == 0).astype(int)

#Direct distance to hydrology
forest['Direct_Distance_To_Hydrology'] = np.sqrt((forest.Vertical_Distance_To_Hydrology**2) + \
    (forest.Horizontal_Distance_To_Hydrology**2)).astype(float).round(2)

column_list = forest.columns.tolist()
column_list = [c for c in column_list if c[:9] != 'Soil_Type']
column_list.insert(10, 'Direct_Distance_To_Hydrology')
column_list.insert(11, 'Elevation_0_2600')
column_list.insert(12, 'Elevation_2600_3100')
column_list.insert(13, 'Elevation_3100_8000')
column_list.insert(14, 'Aspect2')
column_list.insert(15, 'Above_Sealevel')
column_list.insert(16, '3PM_0_Hillshade')
column_list.extend(soil_types)
column_list.extend(['Cover_Type'])
columns = []
for col in column_list:
    if col not in columns:
        if col != 'Cover_Type':
            columns.append(col)
columns.append('Cover_Type')
        
forest = forest[columns]
forest.fillna(0,inplace=True) # Replace nans with 0 for our soil type bins
forest.shape

In [ ]:
forest.head()

In [ ]:
forest.shape

## Add Feature Interactions

In [ ]:
for i in range(forest.shape[1]-1):
    for j in range(54):
        if i != j:
            forest[forest.columns.tolist()[i]+"_"+forest.columns.tolist()[j]] = forest[forest.columns.tolist()[i]]*forest[forest.columns.tolist()[j]]

In [ ]:
forest.shape

## Remove Columns That Have No Value (again)

In [ ]:
forest, removed = removeColumns(forest)

In [ ]:
forest.shape

In [ ]:
forest.head()

# Transform the continuous features
###### We will try Normalization, Standardized Scaling, and MinMax Scaling
###### Note: there is no need to impute any data points as this is a pretty clean data set

In [ ]:
def transformData(df, to_remove):    
    # Initialize lists and variables
    chunk_size = 0.1 #Validation chunk size
    seed = 0 # Use the same random seed to ensure consistent validation chunk usage

    ranks = [] #array of importance rank of all features
    X_all = [] # all features
    X_all_add = [] # Additionally we will make a list of subsets
    rem = [] # columns to be dropped
    i_rem = [] # indexes of columns to be dropped
    trans_list = [] # Transformations
    comb = [] # combinations
    comb.append("All+1.0")

    ratio_list = [0.75,0.50,0.25] #Select top 75%, 50%, 25% of features
    features = [] # feature selection models
    model_features = [] # names of feature selection models

    # reorder the data to have continuous variables come first
    continuous = [] # continuous variables
    categorical = [] # categorical variables
    final_columns = [] # final columns list
    for col in df.columns.tolist():
        if col in to_remove:
            pass
        elif col == 'Cover_Type':
            pass
        elif df[col].nunique() > 4:
            continuous.append(col)
        else:
            categorical.append(col)

    final_columns.extend(continuous)
    final_columns.extend(categorical)
    final_columns.append('Cover_Type')
    df = df[final_columns]
    num_rows, num_cols = df.shape
    cols = df.columns
    size = len(continuous) # Number of continuous columns

    i_cols = []
    for i in range(0,num_cols-1):
        i_cols.append(i)

    # Create the data arrays for model building
    val_array = df.values
    X = val_array[:,0:(num_cols-1)]
    y = val_array[:,(num_cols-1)]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=chunk_size, random_state=seed)
    X_all.append(['Orig','All', X_train,X_val,1.0,cols[:num_cols-1],rem,ranks,i_cols,i_rem])

    # Standardize the data

    X_temp = StandardScaler().fit_transform(X_train[:,0:size])
    X_val_temp = StandardScaler().fit_transform(X_val[:,0:size])

    # Recombine data
    X_con = np.concatenate((X_temp,X_train[:,size:]),axis=1)
    X_val_con = np.concatenate((X_val_temp,X_val[:,size:]),axis=1)

    X_all.append(['StdSca','All', X_con,X_val_con,1.0,cols,rem,ranks,i_cols,i_rem])

    # MinMax Scale the data

    X_temp = MinMaxScaler().fit_transform(X_train[:,0:size])
    X_val_temp = MinMaxScaler().fit_transform(X_val[:,0:size])

    # Recombine data
    X_con = np.concatenate((X_temp,X_train[:,size:]),axis=1)
    X_val_con = np.concatenate((X_val_temp,X_val[:,size:]),axis=1)

    X_all.append(['MinMax', 'All', X_con,X_val_con,1.0,cols,rem,ranks,i_cols,i_rem])

    #Normalize the data

    X_temp = Normalizer().fit_transform(X_train[:,0:size])
    X_val_temp = Normalizer().fit_transform(X_val[:,0:size])

    # Recombine data
    X_con = np.concatenate((X_temp,X_train[:,size:]),axis=1)
    X_val_con = np.concatenate((X_val_temp,X_val[:,size:]),axis=1)

    X_all.append(['Norm', 'All', X_con,X_val_con,1.0,cols,rem,ranks,i_cols,i_rem])

    # Add transformation to the list
    for trans,name,X,X_val,v,cols_list,rem_list,rank_list,i_cols_list,i_rem_list in X_all:
        trans_list.append(trans)
    
    return X_all, y_train

# Create separate datasets for original and engineered data

In [ ]:
X_original_features, y_train = transformData(forest[original_fields], removed)

In [ ]:
X_all_features, y_train = transformData(forest, removed)

# Run Logistic Regression Grid Search 

In [ ]:
model_list = [
    {
        'model':SVC(),
        'params':{ 
            'C':(1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3),
            'kernel':('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'),
            'degree':(1, 2, 3, 4),
            'gamma':('auto', 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3),
            'coef0':(1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3),
            'probability':(True, False),
            'shrinking':(True, False),
            'tol':(1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1),
            'decision_function_shape':('ovo', 'ovr', None)
        }
    },
    {
        'model':LogisticRegression(),
        'params':{ 
            'penalty':('l1', 'l2'), 
            'dual':(True, False), 
            'C':(1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3), 
            'fit_intercept':(True, False), 
            'intercept_scaling':(1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3),
            'max_iter':(1, 10, 100, 1000, 10000),
            'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag'),
            'tol':(1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1),
            'multi_class':('ovr', 'multinomial')
        }
    }
]

In [ ]:
#Run Grid Search over the different data transformations    
def gridSearch(dataset, model, params):
    for trans, s, X, X_val, d, cols, rem, ra, i_cols, i_rem in dataset:
        g = GridSearchCV(model, params, error_score=-999, verbose=3)
        g.fit(X, y_train)
        return trans, g.best_estimator_, g.best_score_, g.best_params_, g.cv_results_

In [ ]:
datasets = [
    {
        'data':X_original_features, 
        'description':'original'
    },
    {
        'data':X_all_features, 
        'description':'all_features'
    }
]

In [ ]:
for m in model_list:
    for d in datasets:
        print (m['model'])
        print (m['params'])
        d['trans'], d['best_estimator'], d['best_score'], d['best_params'], d['cv_results'] = gridSearch(d['data'], m['model'], m['params'])
        print (d['trans'], d['best_estimator'], d['best_score'], d['best_params'], d['cv_results'])